In [ ]:
import polars as pl
import numpy as np
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
import statsmodels.api as sm
import pandas as pd

In [ ]:
# Question 
#1a
df = pl.read_excel("data/fertil2.xlsx")
df = df.select(
    pl.col("children", "age", "educ", "electric", "urban",'spirit','protest','catholic')
)
df =df.with_columns(
    age2=pl.col("age")**2
)
df = df.to_pandas()

In [ ]:
model = smf.ols("children ~ age + age2 + educ + electric + urban", data=df).fit()
print(model.summary())

In [ ]:
model = smf.ols("children ~ age + age2 + educ + electric + urban", data=df).fit(cov_type='HC1')
print(model.summary())

In [ ]:
#It seems that the robust standard errors are generally larger than the non
#robust ones, but not neccesarily always the case.

In [ ]:
#1b
model = smf.ols("children ~ age + age2 + educ + electric + urban + spirit + protest + catholic", data=df).fit()
print(model.summary())
print(model.f_test("spirit = protest = catholic = 0"))

In [ ]:
model = smf.ols("children ~ age + age2 + educ + electric + urban + spirit + protest + catholic", data=df).fit(cov_type='HC1')
print(model.summary())
print(model.f_test("spirit = protest = catholic = 0"))

In [ ]:
#The p-values for the non-robust test is 0.0864, while the p-value for the
#robust test is 0.0911. It seems that robust tests are less likely to report
#something is significant, especially assuming standard errors are greater
#than non-robust ones.

In [ ]:
#1c
df['yhat'] = model.fittedvalues
df["u_hat"] = model.resid
df["u_hat2"] = df["u_hat"]**2
df['yhat2'] = df['yhat']**2
df

In [ ]:
model = smf.ols("u_hat2 ~ yhat + yhat2", data=df).fit()
print(model.summary())
print(model.f_test("yhat = yhat2 = 0"))

In [ ]:
# Question 2 
# A i
df = pl.read_excel("data/Movies.xlsx")
df = df.select(
    pl.all().exclude("month1", "year1"),
    ln_assaults=pl.col("assaults").log(),
    attendance=pl.col("attend_v") + pl.col("attend_m") + pl.col("attend_n")
)
data = df.to_pandas()

In [ ]:
variables  = df.select(pl.all().exclude("assaults","ln_assaults", "^atten.*$", "^h_.*$", "^pr_.*$", "^w.*$")).columns
formula = "ln_assaults ~ " + " + ".join(variables)


model = smf.ols(formula=formula, data=df).fit()
print(model.summary())

In [ ]:
# In comparison to January there seems to be more assaults during late sprin
#and early fall, especially in the summer. (May to Sepetember) So there do
# seem to be some seasonality as assaults are lower during winter months.

In [ ]:
variables  = df.select(pl.all().exclude("assaults","ln_assaults", "^atten.*$", "^h_.*$", "^pr_.*$", "^w.*$")).columns
formula = "attendance ~ " + " + ".join(variables)


model = smf.ols(formula=formula, data=df).fit()
print(model.summary())

In [ ]:
# In comparison to january there seems to be more movie attendance during th
# e summer, especially in June, July, and oddly November. One could argue the
# re is seasonality, but November has an odd peak in movie attendance, which
# makes me think maybe attendance goes along with another variable like relea
# se date for movies.

In [ ]:
#B ii
variables  = df.select(pl.all().exclude("assaults","ln_assaults","wkd_ind", "^pr_.*$", "^atten.*$")).columns
formula = "ln_assaults ~ " + "attend_v + attend_m + attend_n + " + " + ".join(variables) 


model = smf.ols(formula=formula, data=df).fit()
print(model.summary())

In [ ]:
# When taking into account all the controls, it seems that viewing a stro
# ngly violent movie decreases assaults by 0.3 percent, which is significant
# since it's associated p-value is below 0.01.

In [ ]:
#ii
print(model.f_test("attend_v = attend_m = attend_n = 0"))

In [ ]:
# After doing a test to see if the coefficients of strongly violent, mode
# > rately violent, and moderately violent movies have the same affect on assau
# > lts, we fail to reject the null hypothesis that they all are equal. This me
# > ans that they could be different or equal, but we aren't sure. When focusin
# > g on the coefficients and p-values, we see that moderately violent movies a
# > re slightly more significant than the other two movie types, which are simi
# > lar.

In [ ]:
#iii
coeffs = model.params[['attend_v', 'attend_m', 'attend_n']].values
cov = model.cov_params().loc[['attend_v', 'attend_m', 'attend_n'], ['attend_v', 'attend_m', 'attend_n']].values

delta_x = np.array([6, -2, -1]) 

delta_ln_assaults = np.dot(delta_x, coeffs)

std_error = np.sqrt(np.dot(delta_x, np.dot(cov, delta_x)))

lower = delta_ln_assaults - 1.96 * std_error
upper = delta_ln_assaults + 1.96 * std_error

percent_change = 100 * (np.exp(delta_ln_assaults) - 1)
ci_lower = 100 * (np.exp(lower) - 1)
ci_upper = 100 * (np.exp(upper) - 1)

print(f"Predicted % change in assaults: {percent_change:.2f}%")
print(f"95% CI: [{ci_lower:.2f}%, {ci_upper:.2f}%]")

In [ ]:
# C i 
endog = data[['attend_v']]
other_controls  = df.select(pl.all().exclude("assaults", "ln_assaults", "wkd_ind", "^atten.*$", "^pr_.*$")).columns
exog = data[['attend_m', 'attend_n'] + other_controls]
instruments = data[["pr_attend_v", "pr_attend_m", "pr_attend_n"]]
ivolsmod = IV2SLS(dependent=data[["ln_assaults"]], endog=endog, exog=exog, instruments=instruments)
res_ivols = ivolsmod.fit()
print(res_ivols.summary)

In [ ]:
model.cov_params()

In [ ]:
res_ivols.cov

In [ ]:
#ii
coeffs = res_ivols.params[['attend_v', 'attend_m', 'attend_n']].values
cov = res_ivols.cov.loc[['attend_v', 'attend_m', 'attend_n'], ['attend_v', 'attend_m', 'attend_n']].values

delta_x = np.array([6, -2, -1]) 

delta_ln_assaults = np.dot(delta_x, coeffs)

std_error = np.sqrt(np.dot(delta_x, np.dot(cov, delta_x)))

lower = delta_ln_assaults - 1.96 * std_error
upper = delta_ln_assaults + 1.96 * std_error

percent_change = 100 * (np.exp(delta_ln_assaults) - 1)
ci_lower = 100 * (np.exp(lower) - 1)
ci_upper = 100 * (np.exp(upper) - 1)

print(f"Predicted % change in assaults: {percent_change:.2f}%")
print(f"95% CI: [{ci_lower:.2f}%, {ci_upper:.2f}%]")

In [ ]:
coeffs = res_ivols.params[['pr_attend_v', 'pr_attend_m', 'pr_attend_n']].values
cov = model.cov_params().loc[['pr_attend_v', 'pr_attend_m', 'pr_attend_n'], ['pr_attend_v', 'pr_attend_m', 'pr_attend_n']].values

delta_x = np.array([6, -2, -1]) 

delta_ln_assaults = np.dot(delta_x, coeffs)

std_error = np.sqrt(np.dot(delta_x, np.dot(cov, delta_x)))

lower = delta_ln_assaults - 1.96 * std_error
upper = delta_ln_assaults + 1.96 * std_error

percent_change = 100 * (np.exp(delta_ln_assaults) - 1)
ci_lower = 100 * (np.exp(lower) - 1)
ci_upper = 100 * (np.exp(upper) - 1)

print(f"Predicted % change in assaults: {percent_change:.2f}%")
print(f"95% CI: [{ci_lower:.2f}%, {ci_upper:.2f}%]")

In [ ]:
# D 
endog = data[['attend_v']]
other_controls  = df.select(pl.all().exclude("assaults", "ln_assaults", "wkd_ind", "^atten.*$", "^pr_.*$")).columns
exog = data[['attend_m', 'attend_n'] + other_controls] 
instruments = data[["attend_v_f", "attend_m_f", "attend_n_f", "attend_v_b", "attend_m_b", "attend_n_b"]]
ivolsmod = IV2SLS(dependent=data[["ln_assaults"]], endog=endog, exog=exog, instruments=instruments)
res_ivols = ivolsmod.fit()
# sm_ols = results.params
# sm_ols.name = "sm"
# print(pd.concat([res_ivols.params, sm_ols], axis=1))
print(res_ivols.summary)

In [ ]:
coeffs = res_ivols.params[['attend_v', 'attend_m', 'attend_n']].values
cov = res_ivols.cov.loc[['attend_v', 'attend_m', 'attend_n'], ['attend_v', 'attend_m', 'attend_n']].values

delta_x = np.array([6, -2, -1]) 

delta_ln_assaults = np.dot(delta_x, coeffs)

std_error = np.sqrt(np.dot(delta_x, np.dot(cov, delta_x)))

lower = delta_ln_assaults - 1.96 * std_error
upper = delta_ln_assaults + 1.96 * std_error

percent_change = 100 * (np.exp(delta_ln_assaults) - 1)
ci_lower = 100 * (np.exp(lower) - 1)
ci_upper = 100 * (np.exp(upper) - 1)

print(f"Predicted % change in assaults: {percent_change:.2f}%")
print(f"95% CI: [{ci_lower:.2f}%, {ci_upper:.2f}%]")

In [ ]:
# Question 3

df1 = pl.read_excel("data/timeinvar-1.xlsx")
df2 = pl.read_excel("data/timevar-1.xlsx")
df = df1.join(df2, on="id", how="left",validate="1:m")
df